In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [2]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
from keras.models import load_model
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os
import glob

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [3]:
# use csv file to grab images/labels
csv_path = "/home/jovyan/temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")


# Choose column of label for prediction
models = os.listdir("./models/")

mod_menu = {}
for i in range(0, len(models)):
    mod_menu[models[i]] = models[i]

def f(models_menu):
    return models_menu

out2 = interact(f, models_menu=mod_menu)

model = out2.widget.result

interactive(children=(Dropdown(description='models_menu', options={'joeykaminsky2_cnn_Name_25_32.h5': 'joeykam…

In [4]:
#select model
model = out2.widget.result

#get label
def getLabel(model):
    label_header_array1 = model.split("cnn_")
    label_header_array2 = label_header_array1[1].split("_")
    label_header = label_header_array2[0] 
    return label_header

label_header = getLabel(model)                   

In [5]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin

response = requests.get(full_images_location)
soup = BeautifulSoup(response.text, 'html.parser')

image_links = soup.find_all('a', href=True)

image_urls = []

def get_string_before_last_period(input_string):
    parts = input_string.rsplit('.', 1)
    if len(parts) > 1:
        return parts[0]
    else:
        return input_string

images_in_df = list(df["#img"].unique())

for link in image_links:
    image_url = link['href']
    if get_string_before_last_period(image_url) in images_in_df:
    
        if not image_url.startswith(('http:', 'https:')):
            full_image_url = urljoin(full_images_location, image_url)
        else:
            full_image_url = image_url

        image_urls.append(full_image_url)

In [6]:
import os
import cv2
import requests
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

img_list = []

for url in image_urls:
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert the image content to a numpy array
        image_content = np.frombuffer(response.content, np.uint8)
        
        # Decode the image as a BGR image
        im = cv2.imdecode(image_content, cv2.IMREAD_COLOR)
        
        # Resize and convert the image
        im = cv2.resize(im, (28, 28))
        im = img_to_array(im)
        
        # Append the processed image to img_list
        img_list.append(im)
    else:
        print(f"Failed to fetch image from URL: {url}")


In [7]:
# use csv file to grab images/labels
nameCol = df['#img']
predCol = df[label_header]

# init the image suffix, yset, and image list
# Need to set yset and yval_label here

# yset = 
# yval_label = 

In [8]:
# Reshape original input data images for predicting
img_list = np.array(img_list, dtype ="float")/225.0


predictionsMade = []
preds = model.predict(img_list)

# Run all data through the prediction model that was created
for i in range (0,len(img_list)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
    
# Count how many correct predictions were made
correct = 0
print(len(predictionsMade))
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
print("Accuracy: " + str(correct/len(yset)))

AttributeError: 'str' object has no attribute 'predict'

In [37]:
preds

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2]], dtype=float32)

In [38]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value=None,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value=None,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

NameError: name 'yval_label' is not defined

In [39]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred
print(input_text.value)

#Get file path
path = "/home/jovyan/jupyter-suave/temp_csvs/"

# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(os.path.join(path, new_file), index=None)

NameError: name 'input_text' is not defined

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

survey_name = output_text.value

In [ ]:
#Parse url
upload_url = survey_url.split("/main")[0]

if "https" in upload_url:
    upload_url = upload_url.replace("s","",1)
    upload_url = upload_url + ":3001/uploadCSV"    


new_survey_url_base = survey_url.split(user)[0]

In [ ]:
import requests
upload_data = {'name': input_text.value, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = new_survey_url_base + user + "_" + input_text.value + ".csv" + "&views=" + views + "&view=" + view
print(url)
print ("Click the URL to open the new survey")
